In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Data/Listening_Data/songs_listened_to_with_all_info_unique.csv')
df = df.drop('Unnamed: 0.1', axis=1)
df = df.drop('Unnamed: 0', axis = 1)
df.head(2)

,endTime,artistName,trackName,albumName,msPlayed,month,year,Season,trackUri,danceability,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1/20/2022 4:50,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,60,1,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,...,0.0806,0.838,78.743,audio_features,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4
1,1/20/2022 4:53,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,213851,1,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,...,0.0806,0.838,78.743,audio_features,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4


In [3]:
one_hot_df = pd.concat([df.drop('month', axis=1), pd.get_dummies(df['Season'])], axis=1)
one_hot_df.head(2)

,endTime,artistName,trackName,albumName,msPlayed,year,Season,trackUri,danceability,energy,...,id,uri,track_href,analysis_url,duration_ms,time_signature,1,2,3,4
0,1/20/2022 4:50,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,60,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,0.693,...,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4,1,0,0,0
1,1/20/2022 4:53,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,213851,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,0.693,...,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4,1,0,0,0


In [4]:
one_hot_df.columns

Index([         'endTime',       'artistName',        'trackName',
              'albumName',         'msPlayed',             'year',
                 'Season',         'trackUri',     'danceability',
                 'energy',              'key',         'loudness',
                   'mode',      'speechiness',     'acousticness',
       'instrumentalness',         'liveness',          'valence',
                  'tempo',             'type',               'id',
                    'uri',       'track_href',     'analysis_url',
            'duration_ms',   'time_signature',                  1,
                        2,                  3,                  4],
      dtype='object')

In [5]:
df['Count'] = 1
count_df = df.groupby('trackName', as_index = False).sum('Count')
count_df[['trackName', 'Count']]

,trackName,Count
0,#1 Freak (feat. Ty Dolla $ign),9
1,$ave Dat Money (feat. Fetty Wap & Rich Homie Q...,1
2,1 0 0 . m i l ‘ (with Bas),1
3,1-800-273-8255,2
4,10 F*CKS (with Mansa),7
...,...,...
1845,upsides,11
1846,viola,1
1847,way back,7
1848,when the water meets the mountains (acoustic v...,2


In [69]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets

features = one_hot_df[['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 1, 2, 3, 4]]
features.columns = features.columns.astype(str)
labels = pd.merge(one_hot_df, count_df, on='trackName')['Count']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [70]:
labels

0        73
1        73
2        73
3        73
4        73
         ..
14121     3
14122     3
14123     3
14124     1
14125     1
Name: Count, Length: 14126, dtype: int64

In [71]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(features, labels)


RandomForestRegressor(n_estimators=1000, random_state=42)

In [72]:
song_stats = pd.read_csv('Data/Playlist_Data/playlist_stats.csv').drop('Unnamed: 0', axis = 1)
song_stats.head(5)

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,1,2,3,4,trackName
0,0.436,0.655,1,-8.370,0.0583,0.4990,0.000008,0.688,0.412,121.002,1,0,0,0,Good Days
1,0.603,0.784,6,-4.023,0.0620,0.4460,0.000008,0.119,0.769,172.041,1,0,0,0,Dark Red
2,0.913,0.603,8,-4.892,0.2260,0.0272,0.000503,0.167,0.497,123.061,1,0,0,0,Lost
3,0.586,0.893,7,-8.772,0.4390,0.8270,0.642000,0.212,0.650,140.023,1,0,0,0,Funny Thing
4,0.535,0.388,7,-9.805,0.1550,0.5070,0.000000,0.110,0.412,106.703,1,0,0,0,I Hate U


In [73]:
import numpy as np
# Use the forest's predict method on the test data

predictions = rf.predict(song_stats.drop('trackName', axis = 1))
# test_labels = test_labels['Count']

In [74]:
prediction_df = pd.DataFrame()
prediction_df['Count'] = predictions

In [81]:
prediction_df['trackName'] = song_stats['trackName']
prediction_df['color'] = 'pred'
prediction_df.sort_values('Count', ascending=False)

,Count,trackName,color
13,78.000,Strawberry Skies,pred
3,10.931,Funny Thing,pred
0,8.150,Good Days,pred
12,6.817,Super Rich Kids,pred
1,6.664,Dark Red,pred
6,5.533,Love Is Only a Feeling,pred
4,5.450,I Hate U,pred
9,5.415,White Ferrari,pred
11,5.069,Novacane,pred
2,4.310,Lost,pred


In [76]:
original_labels = pd.merge(one_hot_df, count_df, on='trackName')[['Count', 'trackName']]
original_labels['color'] = 'original'
original_labels.head(5)

,Count,trackName,color
0,73,Make the Most (feat. H.E.R.),original
1,73,Make the Most (feat. H.E.R.),original
2,73,Make the Most (feat. H.E.R.),original
3,73,Make the Most (feat. H.E.R.),original
4,73,Make the Most (feat. H.E.R.),original


In [77]:
total_labels = original_labels.append(prediction_df)
total_labels.head(5)

/var/folders/h4/6452mjq15618sb_49rqm1mkw0000gn/T/ipykernel_73799/1640145732.py:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,Count,trackName,color
0,73.0,Make the Most (feat. H.E.R.),original
1,73.0,Make the Most (feat. H.E.R.),original
2,73.0,Make the Most (feat. H.E.R.),original
3,73.0,Make the Most (feat. H.E.R.),original
4,73.0,Make the Most (feat. H.E.R.),original


In [78]:
import plotly.express as px
hovertemplate = "Track: %{customdata[0]} <br> Count: %{customdata[1]} </br>"
fig = px.scatter(total_labels.sort_values('Count'), x = 'Count', y='Count', color='color', custom_data=['trackName', 'Count'])
fig.update_traces(hovertemplate=hovertemplate)
fig.show()

In [20]:
labels = labels.append(prediction_df['Count'])
labels

/var/folders/h4/6452mjq15618sb_49rqm1mkw0000gn/T/ipykernel_73799/1529402440.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labels = labels.append(prediction_df['Count'])


0                                                                73.0
73                                                                2.0
75                                                               51.0
126                                                             146.0
272                                                              95.0
                                          ...                        
12330                                                            82.0
13834                                                            56.0
0                                                              15.094
normalized_count    0        0.392473
73       0.010753
75       0...
0                                                              15.094
Name: Count, Length: 82, dtype: object

In [34]:
labels = labels.drop_duplicates()
labels['normalized_count'] = labels /labels.abs().max()
labels

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [18]:
labels[labels['Count'] == round(predictions[0])]

KeyError: 'Count'